This notebook shows how to retreieve alerts belonging to either a topic (permanent) or query (temporary) archive token. The alerts are then propagated to a sample T0 filter unit.

In [ ]:
import requests, os

In [ ]:
token = os.environ["ARCHIVE_TOKEN"]
header = {"Authorization": "bearer "+token}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'
from ampel.log.AmpelLogger import AmpelLogger
from ampel.ztf.t0.DecentFilter import DecentFilter
from ampel.contrib.hu.t0.XShooterFilter import XShooterFilter

from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

## A. Define filter parameters
We here choose between two filters: DecentFilter, and XShooterFilter (an extension of the former).

In [ ]:
# A basic alert filter with a lot of parameters that can be edited.
DecentFilter??

In [ ]:
# All DecentFilter options + additional parameters for choosing infant, visible transients.
XShooterFilter??

In [ ]:
# Decent filter parameters
filter_config = {
    "min_ndet": 1,          # Min number of detections
    "min_tspan": -1.,        # Min total detection age in alert
    "max_tspan": 5.,        # Max total detection age in alert
    "min_archive_tspan": -1,        # Min total detection age in IPAC DB
    "max_archive_tspan": 5.,        # Max total detection age in IPAC DB
    "min_rb": 0.3,             # real bogus score
    "min_drb": 0.995,         # deep learning real bogus score 
    "max_fwhm": 5.5,        # sexctrator FWHM (assume Gaussian) [pix]
    "max_elong": 1.4,       # Axis ratio of image: aimage / bimage
    "max_magdiff": 1,       # Difference: magap - magpsf [mag]
    "max_nbad": 0,          # number of bad pixels in a 5 x 5 pixel stamp
    "min_sso_dist": 20,     # distance to nearest solar system object [arcsec]
    "min_gal_lat": 14,      # minium distance from galactic plane. Set to negative to disable cut.
    # Gaia rejection (based on catalog matching)
    "gaia_rs": 0,            # Disabled for now!
    "gaia_pm_signif": 3,
    "gaia_plx_signif": 3,
    "gaia_veto_gmag_min": 9,
    "gaia_veto_gmag_max": 20,
    "gaia_excessnoise_sig_max": 999,
    # PS1 rejection (based on alert content)
    "ps1_sgveto_rad": 1,
    "ps1_sgveto_th": 0.8,
    "ps1_confusion_rad": 3,
    "ps1_confusion_sg_tol": 0.1,
}

In [ ]:
# Additional parameters for the Xshooter filter
filter_config.update({
    "max_dec": 999,         # Max declination, shortcuts to transients visible from the south
    "det_within": 1.,     # Recent first detection 
    "ul_within": 1.5,         # A previous upper limit with this time (days)
    # Updated parameters based on infant detections spring 2021. Defaults conservative
    "max_chipsf": 4,        # Best guess value 2
    "max_seeratio": 2,      # Best guess value 1.3
    "min_sumrat": 0.6,      # Best guess value 0.8    min_tspan: -666

} )

## B. Getting a handle to alerts.
A _resume token_, which points to a specific set of alerts. We here present three ways of doing this. 

### 1. Using a query token
In this case a _resume token_ has been produced (and not consumed) e.g. as in `ampel_api_alertquery.ipynb`. 

In [ ]:
# Copy from query
resume_token = ''
# Skip to Sec 3.

### 2. Using a topic token
These are permanent pointers to list of alerts. Here we will first create a stream from this list. 



topic: Y9R82mkcL1SJq16rTt8_vh-_dx0V7rwEHbGgSgUMlqI

description: A list of the final alert of 27 SNe which were detected early during 2020. Created May 24 2022.

topic: fz6nmCJwKSdZXhzm3JuBTlMY3xjPsdhkobVLvZzzmt8

description: A list of 2565 alerts from 27 2020 SNe which were detected early. Created May 24 2022.


In [ ]:
topic_token = "fz6nmCJwKSdZXhzm3JuBTlMY3xjPsdhkobVLvZzzmt8"

In [ ]:
body = {
    "topic": topic_token, 
    "chunk_size": 100
}

In [ ]:
response = requests.post(f"{base_url}/streams/from_topic", headers={"Authorization": "bearer "+token}, json=body )

In [ ]:
if not response.ok:
    print('Accessing stream from topic not successful')
    print(response)

In [ ]:
resume_token = response.json()['resume_token']

In [ ]:
# Alerts are now being queued in the archive, and can be retrieved through this token
resume_token

### 3. Quering random alerts
We will here grab a random set of alerts to check throughput.

In [ ]:
query = {
  "jd": {
    "$gt": 2459500.5,
    "$lt": 2459551.5
  },
  "candidate": {
    "drb": {
      "$gt": 0.995
    },
    "magpsf": {
      "$gt": 16
    },
    "ndethist": {
      "$gt": 0,
      "$lte": 2
    },
  }
}

In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?programid=1'
header = {"Authorization": "bearer "+os.environ["ARCHIVE_TOKEN"]}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
response.ok

In [ ]:
resume_token = response.json()['resume_token']

### 4. Configure and load a filter
As defined above

In [ ]:
logger = AmpelLogger.get_logger()

In [ ]:
t0filter = DecentFilter( **filter_config, logger=logger )
t0filter.post_init()

In [ ]:
t0filter = XShooterFilter( **filter_config, logger=logger )
t0filter.post_init()

## C. Iterate through filters from the stream, checking whether the filter would accept them

In [ ]:
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream": resume_token}   # From above

In [ ]:
alertloader = ZTFArchiveAlertLoader(**config)

In [ ]:
summary = []

In [ ]:
for alert in alertloader.get_alerts():
    #print(alert)
    filter_accept = t0filter.process( ZiAlertSupplier.shape_alert_dict( alert, [] ) )
    summary.append([alert['candid'], alert['objectId'], alert['candidate']['jd'], filter_accept])

    if len(summary) % 100==0:
        print(len(summary))
        print(summary[-1])

In [ ]:
print('Number of unique transients selected:', len( set( [s[1] for s in summary if s[-1]] ) ))

In [ ]:
print('Total number of alertsselected:', len( ( [s[1] for s in summary if s[-1]] ) ) )

In [ ]:
print('Alerts examined', len(summary))

Example based on the filter settings last used above:
- Processing 50 days of alerts: 1096089 alerts pass query candidates, of these yielded 8179 transient selections from 8215 selected alerts (~160/day).
- Processing the 2565 alerts of the 27 2020 infant SNe, 38 alerts of 15 SNe were selected. 